In [8]:
#Lauren Jachimczyk
#Machine Learning in Finance Homework 2

# 1. (5 points) Sign up for your project team.
print("\n 1. Done")

# 2. (15 points) What’s the difference between a Forward contract and a Futures contract?
print("\n 2. The difference between a Forward contract and a Futures contract is that a"
      " Forward contract is OTC and a Futures contract is exchange-traded." 
      "\nA Forward contract is a commitment to purchase at a future date given amount of a commodity or an asset at a price agreed on today."
      "\nA Futures contract is an exchange-traded, standardized forward-like contract that is marked to market daily.")

# 3. (10 points) A company enters into a forward contract to buy 500 barrels of oil in 6 months at
# $75 per barrel. Three months later, the current forward price for oil is $78 per barrel, and the
# risk-free interest rate is 2% per annum. What is the value of the forward contract at this point?

def PV(Pf,rf,m):
      PV=Pf/(1+rf)**(m/12)
      return PV
original_fwd_pv = PV(75,0.02,3) #3 months remaining
print(original_fwd_pv)
current_forward_price=78

Vf = (current_forward_price-original_fwd_pv)*500 #500 barrels
print(f"\n 3. The value of the forward contract is: ${Vf:.2f}")

# 4. (10 points) A wheat farmer expects to harvest 1,000 bushels of wheat in 6 months. To hedge
# against the risk of a decline in wheat prices, the farmer sells 6-month wheat futures contracts at
# $5 per bushel. If the spot price of wheat in 6 months is $4.50 per bushel, how much has the farmer
# saved or lost by hedging?

quantity = 1000
futures_price = 5.0 
spot_price_6mo = 4.5

total_revenue_without_hedge = spot_price_6mo * quantity
total_revenue_with_hedge = futures_price * quantity
hedge_result = total_revenue_with_hedge - total_revenue_without_hedge

print("\n 4.")
print(f"Total revenue without hedge: ${total_revenue_without_hedge}")
print(f"Total revenue with hedge: ${total_revenue_with_hedge}")
print(f"The farmer has {'saved' if hedge_result > 0 else 'lost'} ${abs(hedge_result)} by hedging.")

# 5. (10 points) Suppose the spot price of a commodity is $50, and the forward price for delivery in 1
# year is $55. If the risk-free rate is 8%, is there an arbitrage opportunity? If so, how would you
# exploit it?
import math

spot_price = 50 
forward_price = 55 
risk_free_rate = 0.08 
T = 1  #years

theoretical_forward_price = spot_price * math.exp(risk_free_rate * T)

print("\n 5.")

if theoretical_forward_price < forward_price:
    print("Arbitrage opportunity exists. Strategy: Sell the forward contract, buy the commodity now.")
    arbitrage_profit = forward_price - theoretical_forward_price
    print(f"Arbitrage profit: ${arbitrage_profit:.2f}")
elif theoretical_forward_price > forward_price:
    print("Arbitrage opportunity exists. Strategy: Buy the forward contract, short-sell the commodity now.")
    arbitrage_profit = theoretical_forward_price - forward_price
    print(f"Arbitrage profit: ${arbitrage_profit:.2f}")
else:
    print("No arbitrage opportunity exists.")


 1. Done

 2. The difference between a Forward contract and a Futures contract is that a Forward contract is OTC and a Futures contract is exchange-traded.
A Forward contract is a commitment to purchase at a future date given amount of a commodity or an asset at a price agreed on today.
A Futures contract is an exchange-traded, standardized forward-like contract that is marked to market daily.
74.62961831098825

 3. The value of the forward contract is: $1685.19

 4.
Total revenue without hedge: $4500.0
Total revenue with hedge: $5000.0
The farmer has saved $500.0 by hedging.

 5.
Arbitrage opportunity exists. Strategy: Sell the forward contract, buy the commodity now.
Arbitrage profit: $0.84


In [13]:
# 6. (50 points) The data file ”IR-data.xlsx” contain approximately 11 years of weekly interest rate
# data. Use the data points prior to 2021 to train RNN and LSTM models, respectively, using
# 50 data points as the lag window. Use the trained models to predict the interest rate for each
# test data point in 2021 and beyond. For each model, report the following model parameters and
# outcome:
# (a) The model architecture summary output
# (b) # of epochs trained and final training MSE loss
# (c) For each predicted point, report the forecasted value
# (d) The overall MSE and R2 for the test data

import pandas as pd
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense
from sklearn.metrics import mean_squared_error, r2_score


raw_data = pd.read_excel('IR-data.xlsx')
raw_data['IR'] = pd.to_numeric(raw_data['IR'], errors='coerce')


clean_data = raw_data.dropna(subset=['IR'])
clean_data = clean_data.sort_values(by='DATE').reset_index(drop=True)
clean_data['DATE'] = pd.to_datetime(clean_data['DATE'])
# print(clean_data)

#split into training and testing data
#training data (prior to 2021)
train_data = clean_data[clean_data['DATE'] < '2021-01-01']
#testing data (2021 and beyond)
test_data = clean_data[clean_data['DATE'] >= '2021-01-01']
train_values = train_data['IR'].values
test_values = test_data['IR'].values

#create sequences
seq_length = 50
batch_size = 1

train_generator = TimeseriesGenerator(train_values, train_values, length=seq_length, batch_size=batch_size)
test_generator = TimeseriesGenerator(test_values, test_values, length=seq_length, batch_size=batch_size)

#RNN model
rnn_model = Sequential([
    SimpleRNN(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])

rnn_model.compile(optimizer='adam', loss='mse')
rnn_model.fit(train_generator, epochs=10, verbose=1)
rnn_predictions = rnn_model.predict(test_generator).flatten()

rnn_mse = mean_squared_error(test_values[seq_length:], rnn_predictions)
rnn_r2 = r2_score(test_values[seq_length:], rnn_predictions)

rnn_history = rnn_model.fit(train_generator, epochs=10, verbose=1)
rnn_final_mse = rnn_history.history['loss'][-1] 

#LSTM model
lstm_model = Sequential([
    LSTM(50, activation='relu', input_shape=(seq_length, 1)),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.fit(train_generator, epochs=10, verbose=1)
lstm_predictions = lstm_model.predict(test_generator).flatten()

lstm_mse = mean_squared_error(test_values[seq_length:], lstm_predictions)
lstm_r2 = r2_score(test_values[seq_length:], lstm_predictions)

lstm_history = lstm_model.fit(train_generator, epochs=10, verbose=1)
lstm_final_mse = lstm_history.history['loss'][-1]




Epoch 1/10


/Users/laurenjachimczyk/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/Users/laurenjachimczyk/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.1263
Epoch 2/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0059
Epoch 3/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0050
Epoch 4/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0043
Epoch 5/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0050
Epoch 6/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0048
Epoch 7/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0044
Epoch 8/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0043
Epoch 9/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0040
Epoch 10/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0044
871/871 ━━━━━━━━━━━━━━━━━━━━ 1s 632us/step
Epoch 1/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0043
Epoch 2/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0042
Epoch 3/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - loss: 0.0041
Epoch 4/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0042
E

/Users/laurenjachimczyk/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3411/3411 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 148.0983
Epoch 2/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0120
Epoch 3/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0107
Epoch 4/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0086
Epoch 5/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0083
Epoch 6/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0071
Epoch 7/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0057
Epoch 8/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0052
Epoch 9/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0045
Epoch 10/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0046
871/871 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step
Epoch 1/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0043
Epoch 2/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 0.0047
Epoch 3/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0043
Epoch 4/10
3411/3411 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step 

In [12]:
print("\n 6.")
print("RNN \n A)Model Summary:")
rnn_model.summary()
print(f"\n B)The number of epochs trained is 10 and the final training MSE loss is {rnn_final_mse:.4f}")
print("\n C)Forecasted Values for Each Predicted Point:")
print(rnn_predictions)
print("\n D) Overall MSE: {rnn_mse:.4f}, R2: {rnn_r2:.4f}")

print("\nLSTM \n A)Model Summary:")
lstm_model.summary()
print(f"\n B)The number of epochs trained is 10 and the final training MSE loss is {lstm_final_mse:.4f} ")
print("\n C)")
print(lstm_predictions)
print("\n D) Overall MSE: {lstm_mse:.4f}, R2: {lstm_r2:.4f}")

#creating 1 final dataframe that compares actual interests, RNN predictions and LSTM predictions side by side

print("\n Final Output")
test_dates = test_data['DATE'].iloc[seq_length:].reset_index(drop=True)
test_interest_rates = test_data['IR'].iloc[seq_length:].reset_index(drop=True)

results_df = pd.DataFrame({
    'DATE': test_dates,
    'Actual Interest Rate': test_interest_rates,
    'RNN Predicted Interest Rate': rnn_predictions,
    'LSTM Predicted Interest Rate': lstm_predictions
})

print(results_df)


 6.
RNN 
 A)Model Summary:


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_1 (SimpleRNN)        │ (None, 50)             │         2,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,955 (31.08 KB)

 Trainable params: 2,651 (10.36 KB)

...

 Total params: 31,355 (122.48 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 20,904 (81.66 KB)


 B)The number of epochs trained is 10 and the final training MSE loss is 0.0040 

 C)
[1.6165215 1.6267232 1.6885164 1.7222354 1.6911684 1.639911  1.6237317
 1.6286366 1.6623594 1.7134335 1.72235   1.735008  1.695957  1.7155288
 1.725782  1.6783954 1.6798526 1.6459346 1.6657243 1.6843948 1.6467701
 1.6413921 1.5746073 1.5881091 1.604997  1.584104  1.5731279 1.5707198
 1.5781889 1.5794395 1.6198003 1.6258874 1.6448168 1.6480393 1.632787
 1.6141741 1.5946413 1.5830914 1.5970973 1.6236792 1.6363325 1.6783574
 1.6602882 1.6348909 1.6388001 1.6390151 1.672365  1.6363614 1.6316833
 1.6135805 1.570215  1.5790441 1.603871  1.5839567 1.6140233 1.5935456
 1.6237948 1.570996  1.5714357 1.5379796 1.5087422 1.4634385 1.4708264
 1.5034606 1.5087821 1.5566887 1.5244619 1.4661169 1.4954209 1.4830812
 1.4981331 1.4925523 1.5316563 1.4977185 1.4933915 1.4603026 1.478392
 1.4491628 1.3902758 1.3476992 1.3151839 1.3616915 1.376919  1.4130466
 1.3805552 1.3297819 1.3192358 1.2278898 1.2371702 1.2881259 1.